<a href="https://colab.research.google.com/github/mylethidiem/artificial-intelligence-projects/blob/main/Architecture%20Project%20Gradient%20Vanishing%20in%20MLP/7_GradientNorm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **0. Import libraries**

In [ ]:
import random
import matplotlib.pyplot as plt # truc quan hoa
import numpy as np

import torch
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision.datasets import FashionMNIST #download fashion mnist data


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True

SEED = 42
set_seed(SEED)

## **1. Prepare dataset**


In [ ]:
train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [ ]:
train_ratio = 0.9
# train_size là số lượng mẫu dữ liệu dùng để train (54000 mẫu = 90% của tập train_dataset)
train_size = int(train_ratio * len(train_dataset)) #90%
val_size = len(train_dataset) - train_size #10%

# train_subset là tập dữ liệu con được tạo ra bằng cách chia ngẫu nhiên train_dataset thành 2 phần
# Hàm random_split() chia train_dataset thành 2 tập con:
# - train_subset: chứa train_size mẫu (54000 mẫu)
# - val_subset: chứa val_size mẫu (6000 mẫu)
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

batch_size = 64
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
# Tại sao lại shuffle tập data train?
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(f"Train size: {len(train_subset)}")
print(f"Validation size: {len(val_subset)}")
print(f"Test size: {len(test_dataset)}")

## **2. Build MLP network `with Gradient Norm`**




In [ ]:
class MLP(nn.Module):
  def __init__(self, input_dims, hidden_dims, output_dims):
    super(MLP, self).__init__()
    self.layer1 = nn.Linear(input_dims, hidden_dims)#
    self.layer2 = nn.Linear(hidden_dims, hidden_dims)
    self.layer3 = nn.Linear(hidden_dims, hidden_dims)
    self.layer4 = nn.Linear(hidden_dims, hidden_dims)
    self.layer5 = nn.Linear(hidden_dims, hidden_dims)
    self.layer6 = nn.Linear(hidden_dims, hidden_dims)
    self.layer7 = nn.Linear(hidden_dims, hidden_dims)
    self.output = nn.Linear(hidden_dims, output_dims)

  def forward(self, x):
    x = nn.Flatten()(x)
    x = self.layer1(x)
    x = nn.Sigmoid()(x)
    x = self.layer2(x)
    x = nn.Sigmoid()(x)
    x = self.layer3(x)
    x = nn.Sigmoid()(x)
    x = self.layer4(x)
    x = nn.Sigmoid()(x)
    x = self.layer5(x)
    x = nn.Sigmoid()(x)
    x = self.layer6(x)
    x = nn.Sigmoid()(x)
    x = self.layer7(x)
    x = nn.Sigmoid()(x)
    out = self.output(x)
    return out


In [ ]:
input_dims = 784 #28x28 pixel = 784 pixel
#2^7:giá trị đủ lớn để có thể học đc các đặc trưng phức tạp, không quá lớn để bị overfitting
hidden_dims = 128
output_dims = 10 #10 class
lr = 1e-2
model = MLP(input_dims, hidden_dims, output_dims).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

## **3. Training**

In [ ]:
# Số lượng epochs (chu kỳ huấn luyện) cho quá trình training
# Mỗi epoch, mô hình sẽ được huấn luyện trên toàn bộ tập dữ liệu training một lần
# Với epochs = 100, mô hình sẽ được huấn luyện 100 lần trên tập dữ liệu
epochs = 100
train_loss_lst =[]
train_acc_lst = []
val_loss_lst = []
val_acc_lst = []

# Lặp qua số epochs đã định
for epoch in range(epochs):
  # Khởi tạo các biến để tính loss và accuracy cho tập train
  train_loss = 0
  train_acc = 0
  count = 0
  # Chuyển model sang chế độ train
  model.train()
  # Lặp qua từng batch dữ liệu train
  for X_train, y_train in train_loader:
    # Chuyển dữ liệu sang device (GPU/CPU)
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    # Xóa gradient tích lũy từ bước trước
    optimizer.zero_grad()
    # Forward pass: tính output của model
    outputs = model(X_train)
    # Tính loss giữa output và ground truth
    loss = criterion(outputs, y_train)
    # Backward pass: tính gradient
    loss.backward()
    # Cập nhật trọng số dựa trên gradient
    optimizer.step()
    # Cộng dồn loss và accuracy
    train_loss += loss.item()
    # Tính số lượng dự đoán đúng trong batch hiện tại:
    # 1. torch.argmax(outputs,1): Lấy chỉ số của giá trị lớn nhất trên mỗi hàng (dim=1) của outputs
    #    -> Đây chính là nhãn được dự đoán bởi mô hình
    # 2. == y_train: So sánh với nhãn thực tế, trả về tensor boolean (True nếu dự đoán đúng)
    # 3. .sum(): Đếm số lượng giá trị True (số lượng dự đoán đúng)
    # 4. .item(): Chuyển đổi tensor thành số Python
    # Cộng dồn số lượng dự đoán đúng vào biến train_acc
    train_acc += (torch.argmax(outputs,1) == y_train).sum().item()
    count += len(y_train)
  # Tính trung bình loss và accuracy trên tập train
  train_loss /= len(train_loader)
  train_loss_lst.append(train_loss)
  train_acc /= count
  train_acc_lst.append(train_acc)

  # Khởi tạo các biến để tính loss và accuracy cho tập validation
  val_loss = 0.0
  val_acc = 0.0
  count = 0
  # Chuyển model sang chế độ evaluation
  model.eval()
  # Tắt tính toán gradient khi validate vì:
  # 1. Không cần cập nhật trọng số trong quá trình validation
  # 2. Giúp tăng tốc độ tính toán và tiết kiệm bộ nhớ
  # 3. Đảm bảo kết quả đánh giá nhất quán vì không có sự thay đổi của trọng số
  with torch.no_grad():
    # Lặp qua từng batch dữ liệu validation
    for X_val, y_val in val_loader:
      # Chuyển dữ liệu sang device
      X_val = X_val.to(device)
      y_val = y_val.to(device)
      # Tính output
      outputs = model(X_val)
      # Tính loss
      loss = criterion(outputs, y_val)
      # Cộng dồn loss và accuracy
      val_loss += loss.item()
      val_acc += (torch.argmax(outputs,1) == y_val).sum().item()
       # Train size có thể khác so với tổng số mẫu thực tế dùng(do loại bỏ mẫu lỗi..?)
      count += len(y_val)
  # Tính trung bình loss và accuracy trên tập validation
  val_loss /= len(val_loader)
  val_loss_lst.append(val_loss)
  val_acc /= count # Chia cho số mẫu thực tế đã sử dụng
  val_acc_lst.append(val_acc)

  # In kết quả sau mỗi epoch
  print(f"EPOCH {epoch+1}/{epochs}, Train_Loss: {train_loss:.4f},Train_Acc: {train_acc:.4f}, Validation Loss: {val_loss:.4f} , Val_Acc:{val_acc:.4f}")

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
ax[0, 0].plot(train_loss_lst, color='green')
ax[0, 0].set(xlabel='Epoch', ylabel='Loss')
ax[0, 0].set_title('Training Loss')

ax[0, 1].plot(val_loss_lst, color='orange')
ax[0, 1].set(xlabel='Epoch', ylabel='Loss')
ax[0, 1].set_title('Validation Loss')

ax[1, 0].plot(train_acc_lst, color='green')
ax[1, 0].set(xlabel='Epoch', ylabel='Accuracy')
ax[1, 0].set_title('Training Accuracy')

ax[1, 1].plot(val_acc_lst, color='orange')
ax[1, 1].set(xlabel='Epoch', ylabel='Accuracy')
ax[1, 1].set_title('Validation Accuracy')

plt.show()

## **4. Evaluation**

In [ ]:
# Khởi tạo list rỗng để lưu trữ nhãn thực tế của tập test
# Sau đó sẽ được sử dụng để so sánh với dự đoán của mô hình
test_target = []
test_predict = []
model.eval()
with torch.no_grad():
  for X_test, y_test in test_loader:
    X_test = X_test.to(device)
    y_test = y_test.to(device)
    outputs = model(X_test)

    test_target.append(y_test.cpu())# ko cần tính toán trên GPU nữa
    test_predict.append(outputs.cpu())

# Ghép các batch của nhãn thực tế và dự đoán thành một tensor duy nhất
# Shape: (batch_size, channels, height, width)
# dim=0: ghép theo batch
# dim=1: ghép theo kênh màu
# dim=2: ghép theo chiều cao
# dim=3: ghép theo chiều rộng

# Tensor 2D thông thường:
#dim=0  # Ghép theo hàng (vertically)
#dim=1  # Ghép theo cột (horizontally)

# Tensor nhiều chiều:
#dim=n  # n là chỉ số của chiều muốn ghép (0-based indexing)
test_target = torch.cat(test_target, dim=0) #ghép theo chiều thứ nhất chiều batch(chiều dọc)
test_predict = torch.cat(test_predict, dim=0)

# Tính độ chính xác trên tập test:
# 1. torch.argmax(test_predict,1) - Lấy chỉ số của giá trị lớn nhất trên mỗi hàng (dự đoán của mô hình)
# 2. So sánh với nhãn thực tế (test_target)
# 3. Tính tổng số dự đoán đúng và chia cho tổng số mẫu
test_acc = (torch.argmax(test_predict,1)==test_target).sum().item()/len(test_target)
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
val_label = []
val_predict = []

model.eval()
with torch.no_grad():
  for X_val, y_val in val_loader:
    X_val, y_val = X_val.to(device), y_val.to(device)

    output = model(X_val)

    # transfer to CP, currently it is tensor
    val_label.append(y_val.cpu())
    val_predict.append(output.cpu())

  val_label = torch.cat(val_label, dim=0)
  val_predict = torch.cat(val_predict, dim=0)
  val_acc = (torch.argmax(val_predict, dim=1) == val_label).sum().item()/len(val_label)

print(f"Validation accuracy: {val_acc:.4f}")